<h2> Bookstore Database Tasks</h2>

#### Requirement 1: Create Database and Tables
1. **Database Creation**:
   - Create a database named `Bookstore`.

2. **Tables**:
   - **Authors**: `author_id` (Primary Key), `name`, `nationality`, `birth_year`
   - **Books**: `book_id` (Primary Key), `title`, `author_id` (Foreign Key), `genre`, `published_year`,`price`
   - **Sales**: `sale_id` (Primary Key), `book_id` (Foreign Key), `quantity_sold`

In [1]:
# Step 1: Create database and tables
import sqlite3
import pandas as pd
# Connect to SQLite database
conn = sqlite3.connect('BookstoreDB.db')
cursor = conn.cursor()

# Create authors table
cursor.execute("""CREATE TABLE IF NOT EXISTS Authors (
    author_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    nationality TEXT,
    birth_year INTEGER
);""")

cursor.execute("""CREATE TABLE IF NOT EXISTS Books (
    book_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    price INTEGER NOT NULL,
    author_id INTEGER,
    genre TEXT,
    published_year INTEGER,
    FOREIGN KEY(author_id) REFERENCES Authors(author_id)
);""")

cursor.execute("""CREATE TABLE IF NOT EXISTS Sales (
    sale_id INTEGER PRIMARY KEY,
    book_id INTEGER,
    quantity_sold INTEGER,
    FOREIGN KEY(book_id) REFERENCES Books(book_id)
);""")

conn.commit()


#### Requirement 2: Insert Sample Data

In [2]:
# Step 2: Insert data into the tables


authors = [
    ('George Orwell', 'British', 1903),
    ('J.K. Rowling', 'British', 1965),
    ('Agatha Christie', 'British', 1890),
    ('J.R.R. Tolkien', 'South African', 1892),
    ('Mark Twain', 'American', 1835)
]
cursor.executemany("INSERT INTO Authors ( name, nationality, birth_year) VALUES ( ?, ?, ?);", authors)

# Insert data into Books
# Insert data into Books with consistent number of fields
books = [
    ( '1984', 1, 'Dystopian', 1949, 100),
    ( 'Harry Potter and the Philosopher\'s Stone', 2, 'Fantasy', 1997, 120),
    ( 'Murder on the Orient Express', 3, 'Mystery', 1934, 50),
    ( 'The Hobbit', 4, 'Fantasy', 1937, 60),
    ( 'Adventures of Huckleberry Finn', 5, 'Adventure', 1884, 70),
    ( 'Animal Farm', 1, 'Satire', 1945, 90),  # Added missing price
    ( 'Harry Potter and the Chamber of Secrets', 2, 'Fantasy', 1998, 90),
    ( 'The Fellowship of the Ring', 4, 'Fantasy', 1954, 30),
    ( 'The Two Towers', 4, 'Fantasy', 1954, 40),
    ( 'The Return of the King', 4, 'Fantasy', 1955, 180)
]
cursor.executemany("INSERT INTO Books ( title, author_id, genre, published_year, price) VALUES ( ?, ?, ?, ?, ?);", books)


# Insert data into Sales
sales = [
    ( 1, 50),
    ( 2, 120),
    ( 3, 75),
    ( 4, 30),
    ( 5, 40),
    ( 6, 100),
    ( 7, 80),
    ( 8, 60),
    ( 9, 55),
    ( 10, 45)
]
cursor.executemany("INSERT INTO Sales ( book_id, quantity_sold) VALUES ( ?, ?);", sales)

conn.commit()


# Read the tables 
authors_df = pd.read_sql("SELECT * FROM Authors;", conn)
books_df = pd.read_sql("SELECT * FROM Books;", conn)
sales_df = pd.read_sql("SELECT * FROM Sales;", conn)

# Display the DataFrames to verify
display("Authors Table:")
display(authors_df)

display("Books Table:")
display(books_df)

display("Sales Table:")
display(sales_df)


'Authors Table:'

,author_id,name,nationality,birth_year
0,1,George Orwell,British,1903
1,2,J.K. Rowling,British,1965
2,3,Agatha Christie,British,1890
3,4,J.R.R. Tolkien,South African,1892
4,5,Mark Twain,American,1835


'Books Table:'

,book_id,title,price,author_id,genre,published_year
0,1,1984,100,1,Dystopian,1949
1,2,Harry Potter and the Philosopher's Stone,120,2,Fantasy,1997
2,3,Murder on the Orient Express,50,3,Mystery,1934
3,4,The Hobbit,60,4,Fantasy,1937
4,5,Adventures of Huckleberry Finn,70,5,Adventure,1884
5,6,Animal Farm,90,1,Satire,1945
6,7,Harry Potter and the Chamber of Secrets,90,2,Fantasy,1998
7,8,The Fellowship of the Ring,30,4,Fantasy,1954
8,9,The Two Towers,40,4,Fantasy,1954
9,10,The Return of the King,180,4,Fantasy,1955


'Sales Table:'

,sale_id,book_id,quantity_sold
0,1,1,50
1,2,2,120
2,3,3,75
3,4,4,30
4,5,5,40
5,6,6,100
6,7,7,80
7,8,8,60
8,9,9,55
9,10,10,45


#### Requirement 3: Query and Use SQL Joins

In [3]:
# Step 3: Queries


# Query 1: Retrieve all books and their authors
books_query = """
SELECT B.title AS 'Book Title', A.name AS 'Author Name', B.genre, B.published_year AS 'Published Year'
FROM Books B
INNER JOIN Authors A ON B.author_id = A.author_id;
"""
allBooksWithAuthor = pd.read_sql_query(books_query,conn)
display(allBooksWithAuthor)




# # Query 2: Find total sales per book (i added totl revenue )
total_sales_query = """
SELECT B.title AS 'Book Title', 
       SUM(S.quantity_sold) AS 'Total Sales',
       SUM(S.quantity_sold * B.price) AS 'Total Revenue'
FROM Books B
JOIN Sales S ON B.book_id = S.book_id
GROUP BY B.book_id
ORDER BY SUM(S.quantity_sold) DESC;
"""

# Execute the query and fetch the results
total_sales_revenue_per_book = pd.read_sql_query(total_sales_query, conn)

# Display the result
display(total_sales_revenue_per_book)




# # Query 3: Combine data from Books and Sales
###title	price		genre	published_year
combinetion_query =  """
SELECT B.title AS "Book Title", 
       B.genre AS "Genre", 
       B.price AS "Price", 
       B.published_year AS "Published Year", 
       S.quantity_sold AS "Quantity Sold" 
FROM Books B
JOIN Sales S ON B.book_id = S.book_id;
"""
combinetion_results = pd.read_sql_query(combinetion_query, conn)

# Display the result
display(combinetion_results)


conn.close()

,Book Title,Author Name,genre,Published Year
0,1984,George Orwell,Dystopian,1949
1,Harry Potter and the Philosopher's Stone,J.K. Rowling,Fantasy,1997
2,Murder on the Orient Express,Agatha Christie,Mystery,1934
3,The Hobbit,J.R.R. Tolkien,Fantasy,1937
4,Adventures of Huckleberry Finn,Mark Twain,Adventure,1884
5,Animal Farm,George Orwell,Satire,1945
6,Harry Potter and the Chamber of Secrets,J.K. Rowling,Fantasy,1998
7,The Fellowship of the Ring,J.R.R. Tolkien,Fantasy,1954
8,The Two Towers,J.R.R. Tolkien,Fantasy,1954
9,The Return of the King,J.R.R. Tolkien,Fantasy,1955


,Book Title,Total Sales,Total Revenue
0,Harry Potter and the Philosopher's Stone,120,14400
1,Animal Farm,100,9000
2,Harry Potter and the Chamber of Secrets,80,7200
3,Murder on the Orient Express,75,3750
4,The Fellowship of the Ring,60,1800
5,The Two Towers,55,2200
6,1984,50,5000
7,The Return of the King,45,8100
8,Adventures of Huckleberry Finn,40,2800
9,The Hobbit,30,1800


,Book Title,Genre,Price,Published Year,Quantity Sold
0,1984,Dystopian,100,1949,50
1,Harry Potter and the Philosopher's Stone,Fantasy,120,1997,120
2,Murder on the Orient Express,Mystery,50,1934,75
3,The Hobbit,Fantasy,60,1937,30
4,Adventures of Huckleberry Finn,Adventure,70,1884,40
5,Animal Farm,Satire,90,1945,100
6,Harry Potter and the Chamber of Secrets,Fantasy,90,1998,80
7,The Fellowship of the Ring,Fantasy,30,1954,60
8,The Two Towers,Fantasy,40,1954,55
9,The Return of the King,Fantasy,180,1955,45
